In [59]:
from visdom import Visdom
import numpy as np
import os
import pandas as pd
import matplotlib
# For draw in bg
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# For draw in bg
plt.switch_backend('agg')
from scipy.signal import medfilt
matplotlib.rcParams.update({'font.size': 8})
import seaborn as sns
sns.set(style="darkgrid")

vis=Visdom(env="test")
win=None
dirname='/Users/zsbjltwjj/Desktop/logs/Pendulum-v0-Offline'
targs_names={'MeanRewMBMF-n-1-h-1-e-1-l-1.0-sh-5-me-1': 'MBMF-n-1-h-1-e-1-l-1.0-sh-5-me-1', 'MeanRewMBMF-n-1-h-2-e-1-l-1.0-sh-5-me-1': 'MBMF-n-1-h-2-e-1-l-1.0-sh-5-me-1', 'MeanRewMF': 'MF'}

In [4]:
def load_data_with_transform_best(dirname, targs_names, smooth_max=4):
    df = pd.read_csv(os.path.join(dirname, 'progress.csv'))  
    return _load_data_with_transform_best(df, targs_names, smooth_max, 1)

def _load_data_with_transform_best(df, targs_names, smooth_max, quant):
    targs = (list(targs_names.keys()))
    df = df[targs]
    
    smooth_range = 1 if len(df) <= 20 else smooth_max
    plot_data = [] 

    for targ, name in sorted(targs_names.items()):
        d = df[[targ]].copy(deep=True)
        d['Epoch'] = range(0, len(d))
        d['Epoch'] = d['Epoch'].astype(int)
        d.insert(2, 'type', name)        
        d = d.rename(index=str, columns={targ: 'Reward'})
       
        max_reward_so_far = -INF
        for idx, row in d.iterrows():            
            max_reward_so_far = max(row['Reward'], max_reward_so_far)
            d.at[idx, 'Reward']=max_reward_so_far
        
        d = d[d['Epoch'] % quant == 0]
        d = d.rename(index=str, columns={'Reward': 'Best reward'})
        plot_data.append(d)    
    df = pd.concat(plot_data)
    return df, 'Best reward'

def load_data(dirname, targs_names, smooth_max=4):
    df = pd.read_csv(os.path.join(dirname, 'progress.csv'))

    targs = (list(targs_names.keys()))
    df = df[df[targs].notnull()]
    
    smooth_range = 1 if len(df) <= 20 else smooth_max
    
    plot_data = [] 
    for targ, name in sorted(targs_names.items()):
        d = df[[targ]]
        d['Epoch'] = range(0, len(d))
        d = d.rolling(smooth_range, center=True, min_periods=1).mean()
        d['Epoch'] = d['Epoch'].astype(int)
        d.insert(2, 'type', name)
        d = d.rename(index=str, columns={targ: 'Reward'})
        plot_data.append(d)    
    df = pd.concat(plot_data)
    return df, 'Reward'

def plot(viz, win, dirname, targs_names, smooth=4, opt='each'):
    if opt == 'each': 
        data, y_name = load_data(dirname, targs_names, smooth_max=smooth)        
    elif opt == 'best': 
        data, y_name = load_data_with_transform_best(dirname, targs_names, smooth_max=smooth)

    sns_plot = sns.relplot(x="Epoch", y=y_name, hue='type', kind="line", data=data)
    fig = sns_plot.fig
    plt.title(dirname)
    figname = os.path.join(dirname, 'eval_result.png')
    fig.savefig(figname)
    image = plt.imread(figname)

    # Show it in visdom
    image = np.transpose(image, (2, 0, 1))    
    return viz.image(image, win=win)

In [68]:
df = pd.read_csv(os.path.join(dirname, 'progress.csv'))
targs=(list(targs_names.keys()))
df = df[df[targs].notnull()]
df = df[targs]
df


,MeanRewMBMF-n-1-h-1-e-1-l-1.0-sh-5-me-1,MeanRewMBMF-n-1-h-2-e-1-l-1.0-sh-5-me-1,MeanRewMF
0,31.014267,-27.574438,11.224059
1,-1.122719,-1.838067,28.814795
2,27.674591,6.628013,-22.613283


In [19]:
smooth_range = 1 if len(df) <= 20 else smooth_max

In [66]:
df = pd.read_csv(os.path.join(dirname, 'progress.csv'))
targs=(list(targs_names.keys()))
df = df[df[targs].notnull()]
df


plot_data = []
for targ, name in sorted(targs_names.items()):
        d = df[[targ]].copy(deep=True)
        d['Epoch'] = range(0, len(d))
        d = d.rolling(smooth_range, center=True, min_periods=1).mean()
        d.insert(2, 'type', name)
        d = d.rename(index=str, columns={targ: 'Reward'})
        plot_data.append(d)
plot_data
df = pd.concat(plot_data)
df
sns_plot = sns.relplot(x="Epoch", y="Reward", hue='type', kind="line", data=df)
fig = sns_plot.fig
plt.title(dirname)
figname = os.path.join(dirname, 'eval_result.png')
fig.savefig(figname)

In [80]:
df = pd.read_csv(os.path.join(dirname, 'progress.csv'))
targs=(list(targs_names.keys()))
df = df[df[targs].notnull()]
df = df[targs]
df

plot_data = []
for targ, name in sorted(targs_names.items()):
        d = df[[targ]].copy(deep=True)
        d['Epoch'] = range(0, len(d))
        d['Epoch'] = d['Epoch'].astype(int)
        d.insert(2, 'type', name)        
        d = d.rename(index=str, columns={targ: 'Reward'})
        max_reward_so_far = -1e9
        for idx, row in d.iterrows():            
            max_reward_so_far = max(row['Reward'], max_reward_so_far)
            d.at[idx, 'Reward']=max_reward_so_far
        
        d = d[d['Epoch'] % 2 == 0]
        d = d.rename(index=str, columns={'Reward': 'Best reward'})
        plot_data.append(d)    
df = pd.concat(plot_data)
df
sns_plot = sns.relplot(x="Epoch", y=["Reward",'Best reward'][1], hue='type', kind="line", data=df)
fig = sns_plot.fig
plt.title(dirname)
figname = os.path.join(dirname, 'eval_result.png')
fig.savefig(figname)